In [1]:
import numpy as np
import heavylight

class SimpleModel(heavylight.Model):

    def __init__(self, initial_pols_if: np.ndarray, mortality_rate: float):
        # storage_function determines what gets stored in the results DataFrame
        super().__init__(storage_function=lambda results: np.round(np.sum(results), 3)) # customize how you aggregate results
        self.initial_pols_if = initial_pols_if
        self.mortality_rate = mortality_rate

    def t(self, t):
        return t

    def num_pols_if(self, t):
        if t == 0:
            return self.initial_pols_if
        return self.num_pols_if(t - 1) - self.pols_death(t - 1) # causes exponential time complexity if uncached

    def pols_death(self, t):
        return self.num_pols_if(t) * self.mortality_rate

    def cashflow(self, t):
        return self.num_pols_if(t) * 100

    def v(self, t):
        if t == 0:
            return 1
        return self.v(t - 1) / (1 + self.forward_rate(t))

    def forward_rate(self, t):
        return 0.04

    def pv_cashflow(self, t):
        return self.cashflow(t) * self.v(t)

# start with 10 policies and constant mortality rate of .01
simple_model = SimpleModel(initial_pols_if=np.ones((10,)), mortality_rate=.01)
# run the model for 5 timesteps
simple_model.RunModel(proj_len=5)
# create a dataframe to store results
results = simple_model.ToDataFrame()

In [2]:
results

,t,cashflow,forward_rate,num_pols_if,pols_death,pv_cashflow,v
0,0,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100...",0.04,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.0...","[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100...",1.000000
1,1,"[99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99....",0.04,"[0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.9...","[0.0099, 0.0099, 0.0099, 0.0099, 0.0099, 0.009...","[95.19230769230768, 95.19230769230768, 95.1923...",0.961538
2,2,"[98.00999999999999, 98.00999999999999, 98.0099...",0.04,"[0.9801, 0.9801, 0.9801, 0.9801, 0.9801, 0.980...","[0.009801, 0.009801, 0.009801, 0.009801, 0.009...","[90.61575443786981, 90.61575443786981, 90.6157...",0.924556
3,3,"[97.0299, 97.0299, 97.0299, 97.0299, 97.0299, ...",0.04,"[0.970299, 0.970299, 0.970299, 0.970299, 0.970...","[0.00970299, 0.00970299, 0.00970299, 0.0097029...","[86.25922778220298, 86.25922778220298, 86.2592...",0.888996
4,4,"[96.059601, 96.059601, 96.059601, 96.059601, 9...",0.04,"[0.9605960100000001, 0.9605960100000001, 0.960...","[0.009605960100000002, 0.009605960100000002, 0...","[82.11214952344324, 82.11214952344324, 82.1121...",0.854804
